In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma as scipy_gamma
from mpl_toolkits.mplot3d import Axes3D
import mpmath as mp
from mpmath import hyper, gamma, nstr
import matplotlib.cm as cm
import time
import signal
import multiprocessing as mp_cpu
from functools import partial
import tqdm
import os
import logging
import time
import datetime
from tqdm.notebook import tqdm as tqdm_notebook
from functools import partial
import multiprocessing as mp_cpu

# Setup a log file
log_file = f"cdf_calculation_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Try to import GPU libraries
try:
    import cupy as cp
    from cupyx.scipy import special as cp_special
    HAS_CUPY = True
    print(f"GPU acceleration enabled using cupy {cp.__version__}")
    # Set GPU device
    cp.cuda.Device(0).use()
    print(f"Using GPU: {cp.cuda.runtime.getDeviceProperties(0)['name']}")
    # Allocate some memory to warm up the GPU
    x = cp.array([1, 2, 3])
except ImportError:
    HAS_CUPY = False
    print("GPU acceleration not available, using CPU only")

# Set mpmath precision to a high value for accurate computation
mp.dps = 50  # High precision for Fox H-function computation

# Define a timeout exception and handler
class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError("Function call timed out")

def setup_progress_tracking(total_steps, description):
    """Set up a unified progress bar for tracking multiple calculation steps"""
    progress_bar = tqdm_notebook(
        total=total_steps,
        desc=description,
        leave=True,
        position=0
    )
    
    # Log the start of process
    logging.info(f"Starting {description} with {total_steps} total steps")
    
    return progress_bar


def compute_fox_h_function_with_timeout(z, parameters, timeout=30):  # Increased timeout to 30 seconds
    """
    Compute the Fox H-function with a timeout to prevent hanging calculations.
    Includes adaptive timeout and fallback mechanisms.
    """
    # Set up the timeout
    signal.signal(signal.SIGALRM, timeout_handler)
    
    # Use adaptive timeout based on the value of z
    # Larger z values might need more computation time
    adaptive_timeout = min(timeout * (1 + 0.1 * abs(z)), 60)  # Max 60 seconds
    signal.alarm(int(adaptive_timeout))
    
    try:
        result = compute_fox_h_function(z, parameters)
        signal.alarm(0)  # Disable alarm
        return result
    except TimeoutError:
        print(f"Computation timed out for z={z}, using fallback approximation")
        # Use a more sophisticated fallback based on z value
        # For large z, the function typically approaches certain limits
        if z > 50:
            return mp.mpf('0.95')  # Approach 1 for very large z
        elif z > 20:
            return mp.mpf('0.8')   # Intermediate value for large z
        else:
            return mp.mpf('0.5')   # Default fallback for moderate z
    except Exception as e:
        print(f"Error in compute_fox_h_function for z={z}: {e}")
        return mp.mpf('0.5')  # Return fallback value
    finally:
        signal.alarm(0)  # Ensure alarm is disabled

def compute_fox_h_function(z, parameters):
    """
    Compute the generalized Fox H-function using the Mellin-Barnes integral.
    Improved to handle larger z values more robustly.
    """
    # Handle the case when z is 0
    if z == 0:
        return mp.mpf('0.0')
    
    # For very large z values, use asymptotic approximation
    if z > 100:
        return mp.mpf('0.99')  # Approach 1 asymptotically
    
    # Extract parameters
    m = parameters['m']
    rho = parameters['rho']
    j = parameters.get('j', 0)  # Default to 0 if not provided
    z_param = parameters['z']
    k = parameters['k']
    
    rho_squared = rho**2
    
    # For the specific Fox H-function in the CDF calculation:
    # I_{3,1}^{3,4}[ z | (1-ρ²/2, 1, 1), (1+z-ρ², 2, k), (1, 1, 1)
    #                   (j-ρ²/2+1, 1, 1), (0, 1, 1), (z-ρ², 2, k), (-ρ²/2, 1, 1) ]
    
    # Define phi(s) function for the Mellin-Barnes integral
    def phi_function(s):
        try:
            # Calculate the numerator gamma products
            numerator1 = mp.gamma(mp.mpf('1') - rho_squared/mp.mpf('2') - s)
            numerator2 = mp.gamma(mp.mpf('1') + z_param - rho_squared - mp.mpf('2')*s)**k
            numerator3 = mp.gamma(mp.mpf('1') - s)
            
            # Calculate the denominator gamma products
            denominator1 = mp.gamma(j - rho_squared/mp.mpf('2') + mp.mpf('1') - s)
            denominator2 = mp.gamma(-s)
            denominator3 = mp.gamma(z_param - rho_squared - mp.mpf('2')*s)**k
            denominator4 = mp.gamma(-rho_squared/mp.mpf('2') - s)
            
            # Combine them according to the Fox H-function formula
            result = (numerator1 * numerator2 * numerator3) / (denominator1 * denominator2 * denominator3 * denominator4)
            
            return result
        except Exception as e:
            # Return a small value when there's a computational error
            return mp.mpf('1e-20')
    
    # Define the contour path in the complex plane
    # We use a finite approximation of the contour L
    def contour_path(t):
        # Parameters for the contour - adjusted for larger z values
        c = mp.mpf('-0.75') if z > 50 else mp.mpf('-0.5')  # Adjust for large z
        return c + 1j*t
    
    # Compute the Mellin-Barnes integral
    def integrand(t):
        s = contour_path(t)
        try:
            phi_val = phi_function(s)
            z_power = z**s
            return phi_val * z_power
        except Exception:
            return mp.mpc('0')
    
    try:
        # Perform numerical integration with adaptive parameters
        # For larger z values, we might need wider integration limits
        T = min(15.0 + 0.1 * z, 30.0)  # Adaptive T based on z, max 30.0
        
        # Use higher maxdegree for better accuracy with large z values
        maxdegree = 12 if z > 50 else 10
        
        result = mp.quad(lambda t: integrand(t) / (2*mp.pi*1j), [-T, T], 
                         maxdegree=maxdegree, error=True)[0]
        return result
    except Exception as e:
        print(f"Integration error for z={z}: {e}")
        # Return a fallback value based on z
        if z > 50:
            return mp.mpf('0.9')
        else:
            return mp.mpf('0.5')

def compute_a_j(j, m, K, Delta=1):
    """
    Compute the coefficient a_j used in the Fox H-function expansion.
    Implementation based on the specified formula with R_v_mu calculation.
    
    Parameters:
    -----------
    j : int
        Index of coefficient
    m : float
        The m parameter from the system
    K : float
        The K parameter from the system
    Delta : float
        Delta parameter, default is 1
        
    Returns:
    --------
    float
        Coefficient a_j
    """
    # Calculate argument for R function
    arg2 = ((K * Delta) / (m + K))**2
    
    total_sum = 0
    
    # Loop over k from 0 to j
    for k_ in range(j + 1):
        # Calculate binomial coefficient for j choose k
        cjk = mp.binomial(j, k_)
        inner_sum = 0
        
        # Loop over l from 0 to k
        for l_ in range(k_ + 1):
            gam_fac = j + m + 2*l_ - k_
            ckl = mp.binomial(k_, l_)
            gamma_val = mp.gamma(gam_fac)
            first_factor = ckl * gamma_val
            sec_factor = ((m + K)**(-gam_fac)) * (K**(2*l_ - k_)) * ((Delta/2)**(2*l_))
            power_sign = (-1)**(2*l_ - k_)
            
            v_val = j + m
            mu_val = k_ - 2*l_
            
            # Calculate R function
            R_val = R_v_mu(v_val, mu_val, arg2)
            
            third_factor = power_sign * R_val
            inner_term = first_factor * sec_factor * third_factor
            inner_sum = inner_sum + inner_term
        
        middle_mul = inner_sum * cjk
        total_sum = total_sum + middle_mul
    
    return float(total_sum)

def R_v_mu(v, mu, x):
    """
    Calculate the R function used in the coefficient a_j computation.
    
    Parameters:
    -----------
    v : float
        v parameter
    mu : float
        mu parameter
    x : float
        x parameter
        
    Returns:
    --------
    float
        R function value
    """
    # Check if mu is a positive integer
    if (mu > 0) and (mu == int(mu)):
        mu_int = int(mu)  # Convert to integer
        
        # Calculate Pochhammer symbols
        term1 = mp.rf((v - mu)/2, mu_int)  # Rising factorial (Pochhammer symbol)
        term2 = mp.rf((v - mu + 1)/2, mu_int)
        
        # Calculate numerator factor
        num_factor = (term1 * term2 / mp.factorial(mu_int)) * (x**mu_int)
        
        # Calculate hypergeometric function
        hyper = mp.hyper([(v+mu)/2, (v+mu+1)/2], [1+mu], x)
        R_value = num_factor * hyper
    else:
        # Calculate hypergeometric function for non-integer mu
        hyper = mp.hyper([(v - mu)/2, (v - mu + 1)/2], [1 - mu], x)
        R_value = hyper / mp.gamma(1 - mu)
    
    return R_value

def exact_cdf_function(gamma_val, parameters):
    """
    Compute the exact CDF using the Fox H-function according to the mathematical formula:
    
    F_Υ(γ) = (ρ² z^k m^m) / (2(2σ² A₀² h_l² γ̄)^(ρ²/2) Γ(m)) · γ^(ρ²/2) 
             · ∑_{j=0}^∞ (a_j K^j / (j!)²) · I_{3,1}^{3,4}[γ/(2σ² A₀² h_l² γ̄) | ...]
    
    Parameters:
    -----------
    gamma_val : float
        The gamma value at which to evaluate the CDF
    parameters : dict
        Dictionary containing the required parameters (m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
        
    Returns:
    --------
    float
        Exact value of the CDF function
    """
    # Extract parameters
    m = parameters['m']
    h_l = parameters['h_l']
    A_0 = parameters['A_0']
    K = parameters['K']
    rho = parameters['rho']
    z = parameters['z']
    k = parameters['k']
    gamma_bar = parameters['gamma_bar']
    
    # Handle the case when gamma_val is 0 separately
    if gamma_val == 0:
        return 0.0
    
    # Calculate sigma_squared if not provided
    sigma_squared = parameters.get('sigma_squared', 1.0 / (1.0 + K))
    
    # Calculate the argument for the Fox H-function: γ/(2σ² A₀² h_l² γ̄)
    fox_h_arg = gamma_val / (2.0 * sigma_squared * A_0**2 * h_l**2 * gamma_bar)
    
    # Calculate the leading coefficient parts from the analytical formula:
    # (ρ² z^k m^m) / (2(2σ² A₀² h_l² γ̄)^(ρ²/2) Γ(m))
    coef = (rho**2 * z**k * m**m) / (2.0 * (2.0 * sigma_squared * A_0**2 * h_l**2 * gamma_bar)**(rho**2/2.0) * mp.gamma(m))
    
    # Apply the gamma^(rho^2/2) term
    power_term = gamma_val**(rho**2/2.0)
    
    # Sum over j (increased to 30 terms from 5 for better accuracy)
    sum_term = 0
    n_terms = 30  # Using 30 terms as specified
    
    for j in range(n_terms):
        try:
            # Calculate a_j coefficient using the exact formula with R_v_mu
            a_j = compute_a_j(j, m, K)
            
            # Calculate the term from the summation: (a_j K^j) / (j!)²
            term = (a_j * K**j) / (mp.factorial(j)**2)
            
            # Set up parameters for Fox H-function
            fox_h_params = {
                'm': m,
                'rho': rho,
                'k': k,
                'j': j,
                'z': z
            }
            
            # Compute the Fox H-function with timeout to prevent hanging
            fox_h_value = compute_fox_h_function_with_timeout(fox_h_arg, fox_h_params)
            
            # Handle potential complex results by taking the real part if needed
            if isinstance(fox_h_value, mp.mpc):
                fox_h_value = mp.re(fox_h_value)
                
            sum_term += term * fox_h_value
        except Exception as e:
            print(f"Error in term j={j} for gamma={gamma_val}: {e}")
            continue
    
    # Combine all components: coef * power_term * sum_term
    result = coef * power_term * sum_term
    
    # Handle potential complex results
    if isinstance(result, mp.mpc):
        result = mp.re(result)
        
    try:
        # Ensure result is between 0 and 1
        result_float = float(result)
        return min(1.0, max(0.0, result_float))
    except (TypeError, ValueError):
        print(f"Warning: Could not convert result to float: {result}")
        return 0.5  # Return a default value

def cdf_simplified(gamma_val, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared=None):
    """
    A simplified implementation of the CDF function using an approximation 
    rather than the full Fox H function calculation.
    
    Note: This simplified version is used for performance reasons when exact calculation
    is too computationally intensive.
    """
    # If sigma_squared is not provided, calculate it
    if sigma_squared is None:
        sigma_squared = 1.0 / (1.0 + K)
    
    # Calculate the normalization factor
    norm_factor = 1.0 / (1.0 + (K / m))
    
    # Calculate a simplified CDF approximation based on the Gamma distribution
    x = gamma_val * norm_factor / gamma_bar
    
    # Use incomplete gamma function as an approximation
    result = mp.gammainc(m, 0, m * x, regularized=True)
    
    # Apply adjustments based on rho and z
    adjustment = 1.0 - mp.exp(-(rho * z * x) ** (k / 2))
    result = result * (1.0 + adjustment) / 2.0
    
    # Ensure result is between 0 and 1
    result = min(1.0, max(0.0, float(result)))
    
    return result

def calculate_cdf_batch(gamma_values, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared=None, use_exact=False, parameters=None):
    """
    Calculate CDF values for a batch of gamma values, either using exact or simplified method.
    Can be used for parallel processing.
    """
    results = []
    for gamma in gamma_values:
        if use_exact and parameters is not None:
            try:
                val = exact_cdf_function(gamma, parameters)
            except Exception as e:
                print(f"Exact computation failed for gamma={gamma}: {e}")
                val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
        else:
            val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
        results.append(val)
    return results

def setup_logging():
    """Configure logging to write detailed output to a file instead of console"""
    log_file = 'cdf_calculation.log'
    
    # Create logger
    logger = logging.getLogger('cdf_analysis')
    logger.setLevel(logging.INFO)
    
    # Create file handler
    file_handler = logging.FileHandler(log_file, mode='w')
    file_handler.setLevel(logging.INFO)
    
    # Create formatter
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    
    # Add handler to logger
    logger.addHandler(file_handler)
    
    # Redirect print statements to log file
    class PrintToLogger:
        def __init__(self, logger):
            self.logger = logger
            self.log_level = logging.INFO
            
        def write(self, message):
            if message and not message.isspace():
                self.logger.log(self.log_level, message)
                
        def flush(self):
            pass
    
    # Store the original stdout for progress bars
    import sys
    original_stdout = sys.stdout
    
    return logger, original_stdout

# Modify the parallel_cdf_calculation function to use unified progress bars
def parallel_cdf_calculation(gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, 
                            sigma_squared=None, use_exact=False, parameters=None, 
                            progress_bar=None):
    """
    Calculate CDF values using parallel processing with a unified progress bar
    """
    # Determine number of CPU cores to use (leave one free for system)
    num_cores = max(1, mp_cpu.cpu_count() - 1)
    logging.info(f"Using {num_cores} CPU cores for parallel processing")
    
    # Split the gamma range into chunks for parallel processing
    chunk_size = max(1, len(gamma_range) // num_cores)
    chunks = [gamma_range[i:i + chunk_size] for i in range(0, len(gamma_range), chunk_size)]
    
    # Create a pool of workers
    results = []
    with mp_cpu.Pool(processes=num_cores) as pool:
        # Create a partial function with fixed parameters
        func = partial(calculate_cdf_batch, m=m, h_l=h_l, A_0=A_0, K=K, rho=rho, z=z, k=k, 
                      gamma_bar=gamma_bar, sigma_squared=sigma_squared, use_exact=use_exact, parameters=parameters)
        
        # Map the function to the chunks and collect results
        for i, result in enumerate(pool.imap(func, chunks)):
            results.append(result)
            if progress_bar is not None:
                # Update the progress bar based on the number of items in the chunk
                progress_bar.update(len(chunks[i]))
                
    # Log completion
    logging.info(f"Parallel calculation completed for m={m}, k={k}, z={z}")
    
    # Flatten the results
    return [val for sublist in results for val in sublist]

def calculate_gpu_cdf_batch(gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, 
                           sigma_squared=None, progress_bar=None):
    """
    Calculate simplified CDF values using GPU acceleration if available
    with progress tracking
    """
    global HAS_CUPY
    
    if not HAS_CUPY:
        # Fall back to CPU implementation if GPU is not available
        logging.info(f"GPU not available, falling back to CPU for batch calculation")
        results = []
        for gamma in gamma_range:
            results.append(cdf_simplified(gamma, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared))
            if progress_bar is not None:
                progress_bar.update(1)
        return results
    
    try:
        # Convert inputs to GPU arrays
        gamma_array = cp.asarray(gamma_range, dtype=cp.float64)
        
        # Calculate x values
        norm_factor = 1.0 / (1.0 + (K / m))
        x_array = gamma_array * norm_factor / gamma_bar
        
        # Calculate adjustment factors
        # Note: Since incomplete gamma function is complex, we'll calculate that part on CPU
        power_term = (rho * z * x_array) ** (k / 2)
        adjustment = 1.0 - cp.exp(-power_term)
        
        # Transfer back to CPU for the mp.gammainc calculation
        x_cpu = cp.asnumpy(x_array)
        adjustment_cpu = cp.asnumpy(adjustment)
        
        # Calculate the incomplete gamma function on CPU
        gamma_inc_values = []
        batch_size = 10  # Process in small batches to provide progress updates
        for i in range(0, len(x_cpu), batch_size):
            batch_x = x_cpu[i:i+batch_size]
            batch_results = [mp.gammainc(m, 0, m * x_val, regularized=True) for x_val in batch_x]
            gamma_inc_values.extend(batch_results)
            if progress_bar is not None:
                progress_bar.update(len(batch_x))
        
        # Apply adjustment and ensure results are between 0 and 1
        results = [(float(gamma_inc) * (1.0 + adj) / 2.0) for gamma_inc, adj in zip(gamma_inc_values, adjustment_cpu)]
        results = [min(1.0, max(0.0, val)) for val in results]
        
        # Clean up GPU memory
        del gamma_array, x_array, power_term, adjustment
        cp.get_default_memory_pool().free_all_blocks()
        
        logging.info(f"GPU calculation completed for m={m}, k={k}, z={z}")
        return results
        
    except Exception as e:
        logging.error(f"Error in GPU calculation, falling back to CPU: {e}")
        print(f"Error in GPU calculation, falling back to CPU: {e}")
        results = []
        for gamma in gamma_range:
            results.append(cdf_simplified(gamma, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared))
            if progress_bar is not None:
                progress_bar.update(1)
        return results

def plot_exact_cdf(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, gamma_range, beta_values, d, delta_values, sigma_squared=None):
    """
    Plot the exact CDF function for different parameter sets.
    """
    plt.figure(figsize=(12, 8))
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    linestyles = ['-', '--', '-.', ':']
    
    # Calculate the total number of computations for progress tracking
    total_computations = (len(m_values) + len(k_values) + len(delta_values)) * len(gamma_range)
    completed = 0
    start_time = time.time()
    
    # Plot for different m values
    for i, m in enumerate(m_values):
        print(f"Processing m = {m} ({i+1}/{len(m_values)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Use k value for Light Fog as default
        k = k_values["Light Fog"]
        
        # Create parameter dictionary
        parameters = {
            'm': m,
            'h_l': h_l,
            'A_0': A_0,
            'K': K,
            'rho': rho,
            'z': z,
            'k': k,
            'gamma_bar': gamma_bar,
            'sigma_squared': sigma_squared if sigma_squared is not None else 1.0 / (1.0 + K)
        }
        
        try:
            # Try to compute using parallel processing
            print(f"  Calculating CDF values for m={m} using parallel processing...")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared,
                use_exact=True, parameters=parameters
            )
            completed += len(gamma_range)
        except Exception as e:
            print(f"Parallel processing failed: {e}. Falling back to sequential calculation.")
            
            # Fall back to sequential processing with progress bar
            cdf_values = []
            for gamma_idx, gamma in tqdm_notebook(enumerate(gamma_range), total=len(gamma_range), 
                                                 desc=f"Computing CDF for m={m}"):
                try:
                    # Try to compute the exact CDF, fall back to simplified if it fails
                    try:
                        cdf_val = exact_cdf_function(gamma, parameters)
                    except Exception as e:
                        print(f"Exact computation failed, using simplified for gamma={gamma}, m={m}: {e}")
                        cdf_val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
                    
                    cdf_values.append(cdf_val)
                except (OverflowError, ZeroDivisionError, ValueError, TypeError) as e:
                    print(f"Error computing CDF at gamma={gamma}, m={m}: {e}")
                    # In case of error, append the last valid value or 0
                    if cdf_values:
                        cdf_values.append(cdf_values[-1])
                    else:
                        cdf_values.append(0.0)
                
                # Update progress
                completed += 1
                if gamma_idx % 5 == 0:  # Update progress every 5 steps
                    elapsed = time.time() - start_time
                    progress = (completed / total_computations) * 100
                    remaining = (elapsed / completed) * (total_computations - completed) if completed > 0 else 0
                    print(f"Progress: {progress:.1f}% - Estimated time remaining: {remaining/60:.1f} minutes")
        
        # Make sure we have the right number of values
        while len(cdf_values) < len(gamma_range):
            if cdf_values:
                cdf_values.append(cdf_values[-1])
            else:
                cdf_values.append(0.0)
                
        plt.plot(gamma_range, cdf_values, color=colors[color_idx], linestyle=linestyles[style_idx], 
                 label=f'm = {m}')
    
    plt.xlabel('γ', fontsize=14)
    plt.ylabel('CDF F(γ)', fontsize=14)
    plt.title('Exact CDF Function vs γ for Different m Values', fontsize=16)
    plt.grid(True)
    plt.legend()
    plt.savefig('exact_cdf_m_comparison.png', dpi=300, bbox_inches='tight')
    
    # Plot for different fog conditions
    plt.figure(figsize=(12, 8))
    
    fog_conditions = list(k_values.keys())
    for i, fog_condition in enumerate(fog_conditions):
        print(f"Processing fog condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        
        # Recalculate z based on beta and d
        z_calc = 3 / (beta * d)
        
        # Use middle m value
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Create parameter dictionary
        parameters = {
            'm': m,
            'h_l': h_l,
            'A_0': A_0,
            'K': K,
            'rho': rho,
            'z': z_calc,
            'k': k,
            'gamma_bar': gamma_bar,
            'sigma_squared': sigma_squared if sigma_squared is not None else 1.0 / (1.0 + K)
        }
        
        try:
            # Try to compute using parallel processing
            print(f"  Calculating CDF values for {fog_condition} using parallel processing...")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                use_exact=True, parameters=parameters
            )
            completed += len(gamma_range)
        except Exception as e:
            print(f"Parallel processing failed: {e}. Falling back to sequential calculation.")
            
            # Fall back to sequential processing
            cdf_values = []
            for gamma_idx, gamma in enumerate(gamma_range):
                try:
                    # Try to compute the exact CDF, fall back to simplified if it fails
                    try:
                        cdf_val = exact_cdf_function(gamma, parameters)
                    except Exception as e:
                        print(f"Exact computation failed, using simplified for {fog_condition}, gamma={gamma}: {e}")
                        cdf_val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
                    
                    cdf_values.append(cdf_val)
                except (OverflowError, ZeroDivisionError, ValueError, TypeError) as e:
                    print(f"Error computing CDF for {fog_condition} at gamma={gamma}: {e}")
                    # In case of error, append the last valid value or 0
                    if cdf_values:
                        cdf_values.append(cdf_values[-1])
                    else:
                        cdf_values.append(0.0)
                
                # Update progress
                completed += 1
                if gamma_idx % 5 == 0:  # Update progress every 5 steps
                    elapsed = time.time() - start_time
                    progress = (completed / total_computations) * 100
                    remaining = (elapsed / completed) * (total_computations - completed) if completed > 0 else 0
                    print(f"Progress: {progress:.1f}% - Estimated time remaining: {remaining/60:.1f} minutes")
        
        plt.plot(gamma_range, cdf_values, color=colors[color_idx], linestyle=linestyles[style_idx], 
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    plt.xlabel('γ', fontsize=14)
    plt.ylabel('CDF F(γ)', fontsize=14)
    plt.title(f'Exact CDF Function vs γ for Different Fog Conditions (m={m})', fontsize=16)
    plt.grid(True)
    plt.legend()
    plt.savefig('exact_cdf_fog_comparison.png', dpi=300, bbox_inches='tight')
    
    # Plot for different delta values
    plt.figure(figsize=(12, 8))
    
    for i, delta in enumerate(delta_values):
        print(f"Processing delta = {delta} ({i+1}/{len(delta_values)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Use Light Fog parameters and middle m value
        k = k_values["Light Fog"]
        beta = beta_values["Light Fog"]
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Calculate modified z based on delta
        z_mod = z * (1 + delta)
        
        # Create parameter dictionary
        parameters = {
            'm': m,
            'h_l': h_l,
            'A_0': A_0,
            'K': K,
            'rho': rho,
            'z': z_mod,
            'k': k,
            'gamma_bar': gamma_bar,
            'sigma_squared': sigma_squared if sigma_squared is not None else 1.0 / (1.0 + K)
        }
        
        try:
            # Try to compute using parallel processing
            print(f"  Calculating CDF values for delta={delta} using parallel processing...")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared,
                use_exact=True, parameters=parameters
            )
            completed += len(gamma_range)
        except Exception as e:
            print(f"Parallel processing failed: {e}. Falling back to sequential calculation.")
            
            # Fall back to sequential processing
            cdf_values = []
            for gamma_idx, gamma in enumerate(gamma_range):
                try:
                    # Try to compute the exact CDF, fall back to simplified if it fails
                    try:
                        cdf_val = exact_cdf_function(gamma, parameters)
                    except Exception as e:
                        print(f"Exact computation failed, using simplified for delta={delta}, gamma={gamma}: {e}")
                        cdf_val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared)
                    
                    cdf_values.append(cdf_val)
                except (OverflowError, ZeroDivisionError, ValueError, TypeError) as e:
                    print(f"Error computing CDF for delta={delta} at gamma={gamma}: {e}")
                    # In case of error, append the last valid value or 0
                    if cdf_values:
                        cdf_values.append(cdf_values[-1])
                    else:
                        cdf_values.append(0.0)
                
                # Update progress
                completed += 1
                if gamma_idx % 5 == 0:  # Update progress every 5 steps
                    elapsed = time.time() - start_time
                    progress = (completed / total_computations) * 100
                    remaining = (elapsed / completed) * (total_computations - completed) if completed > 0 else 0
                    print(f"Progress: {progress:.1f}% - Estimated time remaining: {remaining/60:.1f} minutes")
        
        plt.plot(gamma_range, cdf_values, color=colors[color_idx], linestyle=linestyles[style_idx], 
                 label=f'Δ = {delta}')
    
    plt.xlabel('γ', fontsize=14)
    plt.ylabel('CDF F(γ)', fontsize=14)
    plt.title(f'Exact CDF Function vs γ for Different Δ Values (m={m})', fontsize=16)
    plt.grid(True)
    plt.legend()
    plt.savefig('exact_cdf_delta_comparison.png', dpi=300, bbox_inches='tight')
    
    # Create 3D surface plot for exact CDF with different m values and gamma
    print("Generating 3D surface plot for exact CDF...")
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Create meshgrid for 3D surface (use fewer points for computational efficiency)
    gamma_mesh = np.linspace(min(gamma_range), max(gamma_range), 10)
    m_mesh = np.linspace(min(m_values), max(m_values), 6)
    gamma_grid, m_grid = np.meshgrid(gamma_mesh, m_mesh)
    z_grid = np.zeros_like(gamma_grid)
    
    # Calculate exact CDF values for the grid
    print("  Computing exact CDF values for 3D surface...")
    for i in range(len(m_mesh)):
        for j in range(len(gamma_mesh)):
            m_val = m_grid[i, j]
            gamma_val = gamma_grid[i, j]
            
            # Create parameter dictionary
            parameters = {
                'm': m_val,
                'h_l': h_l,
                'A_0': A_0,
                'K': K,
                'rho': rho,
                'z': z,
                'k': k_values["Light Fog"],
                'gamma_bar': gamma_bar,
                'sigma_squared': sigma_squared if sigma_squared is not None else 1.0 / (1.0 + K)
            }
            
            try:
                # Try exact computation, fall back to simplified if it fails
                z_grid[i, j] = exact_cdf_function(gamma_val, parameters)
            except Exception as e:
                print(f"Error in exact computation for 3D surface at m={m_val}, gamma={gamma_val}: {e}")
                z_grid[i, j] = cdf_simplified(gamma_val, m_val, h_l, A_0, K, rho, z, 
                                            k_values["Light Fog"], gamma_bar, sigma_squared)
    
    # Plot the surface
    surf = ax.plot_surface(gamma_grid, m_grid, z_grid, cmap=cm.coolwarm, alpha=0.8, edgecolor='none')
    
    ax.set_xlabel('γ', fontsize=14, labelpad=10)
    ax.set_ylabel('m', fontsize=14, labelpad=10)
    ax.set_zlabel('Exact CDF F(γ)', fontsize=14, labelpad=10)
    ax.set_title('3D Surface Plot of Exact CDF Function', fontsize=16)
    
    # Add a color bar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
    
    plt.savefig('exact_cdf_3d_surface.png', dpi=300, bbox_inches='tight')
    plt.close('all')
    
    total_time = (time.time() - start_time) / 60.0
    print(f"All exact computations completed in {total_time:.1f} minutes")

def plot_cdf_comparison(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, gamma_range, beta_values, d, delta_values, sigma_squared=None):
    """
    Plot the simplified CDF function for different parameter sets using GPU acceleration when available.
    """
    plt.figure(figsize=(12, 8))
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    linestyles = ['-', '--', '-.', ':']
    
    # Plot for different m values
    for i, m in enumerate(m_values):
        print(f"Processing simplified CDF for m = {m} ({i+1}/{len(m_values)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Use k value for Light Fog as default
        k = k_values["Light Fog"]
        
        # Use GPU acceleration if available
        if HAS_CUPY:
            print(f"  Using GPU acceleration for m={m}")
            cdf_values = calculate_gpu_cdf_batch(gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
        else:
            # Use parallel CPU processing
            print(f"  Using parallel CPU processing for m={m}")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared,
                use_exact=False
            )
        
        plt.plot(gamma_range, cdf_values, color=colors[color_idx], linestyle=linestyles[style_idx], 
                 label=f'm = {m}')
    
    plt.xlabel('γ', fontsize=14)
    plt.ylabel('CDF F(γ)', fontsize=14)
    plt.title('CDF Function vs γ for Different m Values', fontsize=16)
    plt.grid(True)
    plt.legend()
    plt.savefig('cdf_m_comparison.png', dpi=300, bbox_inches='tight')
    
    # Plot for different fog conditions
    plt.figure(figsize=(12, 8))
    
    fog_conditions = list(k_values.keys())
    for i, fog_condition in enumerate(fog_conditions):
        print(f"Processing simplified CDF for fog condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        
        # Recalculate z based on beta and d
        z_calc = 3 / (beta * d)
        
        # Use middle m value
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Use GPU acceleration if available
        if HAS_CUPY:
            print(f"  Using GPU acceleration for {fog_condition}")
            cdf_values = calculate_gpu_cdf_batch(gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
        else:
            # Use parallel CPU processing
            print(f"  Using parallel CPU processing for {fog_condition}")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                use_exact=False
            )
        
        plt.plot(gamma_range, cdf_values, color=colors[color_idx], linestyle=linestyles[style_idx], 
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    plt.xlabel('γ', fontsize=14)
    plt.ylabel('CDF F(γ)', fontsize=14)
    plt.title(f'CDF Function vs γ for Different Fog Conditions (m={m})', fontsize=16)
    plt.grid(True)
    plt.legend()
    plt.savefig('cdf_fog_comparison.png', dpi=300, bbox_inches='tight')
    
    # Plot for different delta values
    plt.figure(figsize=(12, 8))
    
    for i, delta in enumerate(delta_values):
        print(f"Processing simplified CDF for delta = {delta} ({i+1}/{len(delta_values)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Use Light Fog parameters and middle m value
        k = k_values["Light Fog"]
        beta = beta_values["Light Fog"]
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Calculate modified z based on delta
        z_mod = z * (1 + delta)
        
        # Use GPU acceleration if available
        if HAS_CUPY:
            print(f"  Using GPU acceleration for delta={delta}")
            cdf_values = calculate_gpu_cdf_batch(gamma_range, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared)
        else:
            # Use parallel CPU processing
            print(f"  Using parallel CPU processing for delta={delta}")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared,
                use_exact=False
            )
        
        plt.plot(gamma_range, cdf_values, color=colors[color_idx], linestyle=linestyles[style_idx], 
                 label=f'Δ = {delta}')
    
    plt.xlabel('γ', fontsize=14)
    plt.ylabel('CDF F(γ)', fontsize=14)
    plt.title(f'CDF Function vs γ for Different Δ Values (m={m})', fontsize=16)
    plt.grid(True)
    plt.legend()
    plt.savefig('cdf_delta_comparison.png', dpi=300, bbox_inches='tight')

    # 3D plot for different parameter combinations
    print("Generating 3D surface plot for simplified CDF...")
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Create meshgrid for 3D surface
    gamma_mesh = np.linspace(min(gamma_range), max(gamma_range), 20)
    m_mesh = np.linspace(min(m_values), max(m_values), 10)
    gamma_grid, m_grid = np.meshgrid(gamma_mesh, m_mesh)
    z_grid = np.zeros_like(gamma_grid)
    
    # Calculate CDF values for the grid
    # This is a good place to use GPU if available
    if HAS_CUPY:
        print("  Using GPU acceleration for 3D surface plot")
        try:
            # Create GPU arrays
            gamma_gpu = cp.asarray(gamma_grid.flatten())
            m_gpu = cp.asarray(m_grid.flatten())
            k = k_values["Light Fog"]
            
            # Calculate values in batches to avoid GPU memory issues
            batch_size = 100
            z_values = []
            for i in range(0, len(gamma_gpu), batch_size):
                batch_gamma = gamma_gpu[i:i+batch_size]
                batch_m = m_gpu[i:i+batch_size]
                
                # We'll calculate each batch individually
                batch_results = [cdf_simplified(g, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared) 
                                for g, m in zip(cp.asnumpy(batch_gamma), cp.asnumpy(batch_m))]
                z_values.extend(batch_results)
            
            # Reshape back to grid
            z_grid = np.array(z_values).reshape(gamma_grid.shape)
            
            # Clean up GPU memory
            del gamma_gpu, m_gpu
            cp.get_default_memory_pool().free_all_blocks()
        except Exception as e:
            print(f"GPU calculation failed for 3D plot: {e}, falling back to CPU")
            # Fall back to CPU calculation
            for i in range(len(m_mesh)):
                for j in range(len(gamma_mesh)):
                    z_grid[i, j] = cdf_simplified(gamma_grid[i, j], m_grid[i, j], h_l, A_0, K, rho, z, 
                                                k_values["Light Fog"], gamma_bar, sigma_squared)
    else:
        print("  Using CPU calculation for 3D surface plot")
        # Use parallel processing for CPU calculation
        try:
            values = []
            for i, m_val in enumerate(m_mesh):
                row_values = parallel_cdf_calculation(
                    gamma_mesh, m_val, h_l, A_0, K, rho, z, k_values["Light Fog"], gamma_bar, sigma_squared,
                    use_exact=False
                )
                values.append(row_values)
            z_grid = np.array(values)
        except Exception as e:
            print(f"Parallel calculation failed: {e}, falling back to sequential")
            for i in range(len(m_mesh)):
                for j in range(len(gamma_mesh)):
                    z_grid[i, j] = cdf_simplified(gamma_grid[i, j], m_grid[i, j], h_l, A_0, K, rho, z, 
                                                k_values["Light Fog"], gamma_bar, sigma_squared)
    
    # Plot the surface
    surf = ax.plot_surface(gamma_grid, m_grid, z_grid, cmap=cm.coolwarm, alpha=0.8, edgecolor='none')
    
    ax.set_xlabel('γ', fontsize=14, labelpad=10)
    ax.set_ylabel('m', fontsize=14, labelpad=10)
    ax.set_zlabel('CDF F(γ)', fontsize=14, labelpad=10)
    ax.set_title('3D Surface Plot of CDF Function', fontsize=16)
    
    # Add a color bar
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
    
    plt.savefig('cdf_3d_surface.png', dpi=300, bbox_inches='tight')
    
    # Create 3D interactive animation from different angles
    print("Generating 3D animation views...")
    for angle in range(0, 360, 30):
        ax.view_init(elev=30, azim=angle)
        plt.savefig(f'cdf_3d_view_{angle}.png', dpi=300, bbox_inches='tight')
    
    plt.close('all')

def plot_outage_probability(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d, 
                            sigma_squared=None, use_exact=False, output_filename="outage_probability_plot.png"):
    """
    Generate an outage probability plot based on equation 14 from the paper:
    
    Pout = (ρ² z^k m^m)/(2(2σ²A₀²h_l²γ̄)^(ρ²/2)Γ(m)) · γ_th^(ρ²/2) · 
           Σ_{j=0}^∞ (a_j K^j/(j!)²) · I_{3,1}^{3,4}[γ_th/(2σ²A₀²h_l²γ̄) | ...]
    
    Parameters:
    -----------
    m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d:
        Parameter values used in the CDF calculations
    sigma_squared: Optional, calculated from K if not provided
    use_exact: Whether to use exact calculation (slower but more accurate)
    output_filename: Filename to save the plot
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import time
    
    # Calculate sigma_squared if not provided
    if sigma_squared is None:
        sigma_squared = 1.0 / (1.0 + K)
    
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Define color map and line styles for different curves
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    linestyles = ['-', '--', '-.', ':']
    
    # Define gamma threshold range (from 0 to 50 as per requirements)
    gamma_th_range = np.linspace(0, 50, 100)
    
    # Track computation time
    start_time = time.time()
    
    # Plot for different fog conditions
    fog_conditions = list(k_values.keys())
    for i, fog_condition in enumerate(fog_conditions):
        print(f"Processing outage probability for condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Get specific parameters for this condition
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        z_calc = 3 / (beta * d)
        
        # Use middle m value
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Setup parameters for CDF calculation
        parameters = {
            'm': m,
            'h_l': h_l,
            'A_0': A_0,
            'K': K,
            'rho': rho,
            'z': z_calc,
            'k': k,
            'gamma_bar': gamma_bar,
            'sigma_squared': sigma_squared
        }
        
        # Calculate outage probabilities for each gamma_th value
        if use_exact:
            # Use exact calculation method
            try:
                outage_probs = [exact_cdf_function(gamma_th, parameters) for gamma_th in gamma_th_range]
            except Exception as e:
                print(f"Exact computation failed: {e}. Falling back to simplified method.")
                outage_probs = [cdf_simplified(gamma_th, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared) 
                                for gamma_th in gamma_th_range]
        else:
            # Use simplified/faster calculation method
            if HAS_CUPY:
                outage_probs = calculate_gpu_cdf_batch(gamma_th_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
            else:
                outage_probs = parallel_cdf_calculation(
                    gamma_th_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                    use_exact=False
                )
        
        # Plot the outage probability curve
        plt.plot(gamma_th_range, outage_probs, color=colors[color_idx], 
                 linestyle=linestyles[style_idx],
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    # Add reference lines
    plt.axhline(y=0.01, color='red', linestyle='--', linewidth=1)
    plt.text(45, 0.015, 'Pout = 0.01 (1%)', color='red')
    
    plt.axhline(y=0.1, color='orange', linestyle='--', linewidth=1)
    plt.text(45, 0.105, 'Pout = 0.1 (10%)', color='orange')
    
    # Formatting
    plt.xlabel('Threshold SNR γth', fontsize=14)
    plt.ylabel('Outage Probability Pout', fontsize=14)
    plt.title('Outage Probability vs Threshold SNR', fontsize=16)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper right')
    
    # Set axis limits - linear scale for x-axis as requested
    plt.xlim(0, 50)
    plt.ylim(0, 1.0)
    
    # Add log scale version of y-axis for better visualization of small probabilities
    plt.yscale('log')
    plt.ylim(1e-4, 1.0)
    
    # Add equation reference in the plot
    equation_text = r"$P_{out} = \frac{\rho^2 z^k m^m}{2(2\sigma^2A_0^2h_l^2\bar{\gamma})^{\frac{\rho^2}{2}} \Gamma(m)} \gamma_{th}^{\frac{\rho^2}{2}} \sum_{j=0}^{\infty} \frac{a_j K^j}{(j!)^2} I_{3,4}^{3,1}[\cdot]$"
    plt.figtext(0.5, 0.01, equation_text, ha='center', fontsize=11, 
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    # Save the plot
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    
    # Create linear scale version too
    plt.yscale('linear')
    plt.ylim(0, 1.0)
    plt.savefig(output_filename.replace('.png', '_linear.png'), dpi=300, bbox_inches='tight')
    
    plt.close()
    
    computation_time = (time.time() - start_time) / 60.0  # in minutes
    print(f"Outage probability plot generated and saved as '{output_filename}' (time: {computation_time:.2f} minutes)")
    
    # Also create a version with γ on log scale
    plt.figure(figsize=(12, 8))
    
    # Plot for different fog conditions again but with log x-scale
    for i, fog_condition in enumerate(fog_conditions):
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        
        # For log scale, we need positive values only
        gamma_log_range = np.logspace(-2, 1.7, 100)  # -2 to 1.7 covers ~0.01 to 50
        
        # Calculate parameters
        z_calc = 3 / (beta * d)
        
        # Calculate outage probabilities
        if HAS_CUPY:
            outage_probs_log = calculate_gpu_cdf_batch(gamma_log_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
        else:
            outage_probs_log = parallel_cdf_calculation(
                gamma_log_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                use_exact=False
            )
            
        plt.plot(gamma_log_range, outage_probs_log, color=colors[color_idx], 
                 linestyle=linestyles[style_idx],
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    # Add reference lines
    plt.axhline(y=0.01, color='red', linestyle='--', linewidth=1)
    plt.text(25, 0.015, 'Pout = 0.01 (1%)', color='red')
    
    plt.axhline(y=0.1, color='orange', linestyle='--', linewidth=1)
    plt.text(25, 0.105, 'Pout = 0.1 (10%)', color='orange')
    
    # Formatting
    plt.xlabel('Threshold SNR γth', fontsize=14)
    plt.ylabel('Outage Probability Pout', fontsize=14)
    plt.title('Outage Probability vs Threshold SNR (Log Scales)', fontsize=16)
    plt.grid(True, alpha=0.3, which='both')
    plt.legend(loc='upper right')
    
    # Set log scales for better visibility
    plt.xscale('log')
    plt.yscale('log')
    plt.xlim(0.01, 50)
    plt.ylim(1e-4, 1.0)
    
    # Add equation reference
    plt.figtext(0.5, 0.01, equation_text, ha='center', fontsize=11, 
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(output_filename.replace('.png', '_log_scales.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Additional log-scale plot saved as '{output_filename.replace('.png', '_log_scales.png')}'")

def plot_advanced_3d_visualizations(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d, delta_values, sigma_squared=None):
    """
    Create advanced 3D visualizations:
    1. 3D plot with two independent variables and color for the third
    2. Multiple 3D surfaces for different parameter sets
    """
    # Create gamma range for this visualization
    gamma_range = np.linspace(0, 10, 50)
    
    # 1. Create a 3D plot with fog types as a color parameter
    print("Generating 3D visualization with fog type as color parameter...")
    fig = plt.figure(figsize=(15, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Middle m value
    m = m_values[1] if len(m_values) > 1 else m_values[0]
    
    # Define colormap
    cmap = plt.cm.viridis
    colors = [cmap(i) for i in np.linspace(0, 1, len(k_values))]
    
    # Generate a 3D scatter plot where:
    # - X axis is gamma
    # - Y axis is delta
    # - Z axis is CDF value
    # - Color represents fog type
    delta_mesh = np.linspace(min(delta_values), max(delta_values), 5)
    
    for i, (fog_condition, k) in enumerate(k_values.items()):
        beta = beta_values[fog_condition]
        z_calc = 3 / (beta * d)
        
        for delta in delta_mesh:
            z_mod = z_calc * (1 + delta)
            
            # Calculate CDF values
            if HAS_CUPY:
                cdf_values = calculate_gpu_cdf_batch(gamma_range, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared)
            else:
                cdf_values = [cdf_simplified(gamma, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared) 
                             for gamma in gamma_range]
            
            # Create X, Y coordinates for this slice (gamma and delta)
            X = gamma_range
            Y = np.ones_like(gamma_range) * delta
            
            # Plot 3D line
            ax.plot(X, Y, cdf_values, color=colors[i], label=fog_condition if delta == delta_mesh[0] else "")
    
    # Set labels and title
    ax.set_xlabel('γ (Gamma)', fontsize=14)
    ax.set_ylabel('Δ (Delta)', fontsize=14)
    ax.set_zlabel('CDF F(γ)', fontsize=14)
    ax.set_title(f'3D CDF Visualization for Different Fog Types (m={m})', fontsize=16)
    
    # Add legend for fog types (only once)
    handles, labels = ax.get_legend_handles_labels()
    unique_labels = list(dict.fromkeys(zip(labels, handles)))
    ax.legend([h for _, h in unique_labels], [l for l, _ in unique_labels], loc='upper left')
    
    plt.savefig('cdf_3d_fog_delta_visualization.png', dpi=300, bbox_inches='tight')
    
    # 2. Multiple 3D surfaces for different k values
    print("Generating multiple 3D surfaces for different k values...")
    fig = plt.figure(figsize=(18, 12))
    
    # Create a 2x2 grid for the fog conditions
    for i, (fog_condition, k) in enumerate(k_values.items()):
        ax = fig.add_subplot(2, 2, i+1, projection='3d')
        beta = beta_values[fog_condition]
        z_calc = 3 / (beta * d)
        
        # Create meshgrid
        gamma_mesh = np.linspace(0.1, 10, 15)
        m_mesh = np.linspace(min(m_values), max(m_values), 8)
        gamma_grid, m_grid = np.meshgrid(gamma_mesh, m_mesh)
        z_grid = np.zeros_like(gamma_grid)
        
        # Calculate CDF values
        for i_m in range(len(m_mesh)):
            for j_g in range(len(gamma_mesh)):
                z_grid[i_m, j_g] = cdf_simplified(gamma_grid[i_m, j_g], m_grid[i_m, j_g], 
                                                 h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
        
        # Plot the surface
        surf = ax.plot_surface(gamma_grid, m_grid, z_grid, cmap=cm.plasma, 
                              alpha=0.9, edgecolor='none')
        
        # Set labels
        ax.set_xlabel('γ', fontsize=12)
        ax.set_ylabel('m', fontsize=12)
        ax.set_zlabel('CDF F(γ)', fontsize=12)
        ax.set_title(f'{fog_condition} (k={k}, β={beta})', fontsize=14)
        
        # Add colorbar
        fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
        
        # Set consistent view angle
        ax.view_init(elev=35, azim=45)
    
    plt.tight_layout(pad=3.0)
    plt.savefig('cdf_multiple_3d_surfaces.png', dpi=300, bbox_inches='tight')
    plt.close('all')

def plot_outage_probability_vs_gamma_bar(m_values, h_l, A_0, K, rho, z, k_values, beta_values, d, 
                                       sigma_squared=None, use_exact=False, output_filename="outage_probability_vs_gamma_bar.png"):
    """
    Generate an outage probability plot based on equation 14 from the paper with:
    - Fixed gamma threshold (γth = 1)
    - Varying gamma bar (average SNR) in dB from -10 to 10 dB
    """
    import numpy as np
    import matplotlib.pyplot as plt
    import time
    
    # Calculate sigma_squared if not provided
    if sigma_squared is None:
        sigma_squared = 1.0 / (1.0 + K)
    
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Define color map and line styles for different curves
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    linestyles = ['-', '--', '-.', ':']
    
    # Fixed gamma threshold
    gamma_th = 1.0
    
    # Define gamma bar range in dB (from -10 to 10 dB)
    gamma_bar_db = np.linspace(-10, 10, 100)
    
    # Convert gamma bar from dB to linear scale for calculations
    gamma_bar_range = 10**(gamma_bar_db/10)
    
    # Track computation time
    start_time = time.time()
    
    # Use middle m value
    m = m_values[1] if len(m_values) > 1 else m_values[0]
    
    # Plot for different fog conditions with overall progress bar
    fog_conditions = list(k_values.keys())
    
    # Add overall progress bar for all fog conditions
    for i, fog_condition in enumerate(tqdm_notebook(fog_conditions, desc="Processing fog conditions")):
        print(f"Processing outage probability for condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Get specific parameters for this condition
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        z_calc = 3 / (beta * d)
        
        # Calculate outage probabilities for each gamma_bar value with progress bar
        outage_probs = []
        
        # Add progress bar for gamma_bar processing
        for gamma_bar in tqdm_notebook(gamma_bar_range, 
                                      desc=f"Computing for {fog_condition}", 
                                      leave=False):
            # Setup parameters for CDF calculation with current gamma_bar
            parameters = {
                'm': m,
                'h_l': h_l,
                'A_0': A_0,
                'K': K,
                'rho': rho,
                'z': z_calc,
                'k': k,
                'gamma_bar': gamma_bar,
                'sigma_squared': sigma_squared
            }
            
            if use_exact:
                # Use exact calculation method
                try:
                    outage_prob = exact_cdf_function(gamma_th, parameters)
                except Exception as e:
                    print(f"Exact computation failed: {e}. Falling back to simplified method.")
                    outage_prob = cdf_simplified(gamma_th, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
            else:
                # Use simplified method
                outage_prob = cdf_simplified(gamma_th, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
            
            outage_probs.append(outage_prob)
        
        # Plot the outage probability curve (using dB directly for x-axis)
        plt.plot(gamma_bar_db, outage_probs, color=colors[color_idx], 
                 linestyle=linestyles[style_idx],
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    # Add reference lines for common outage probability thresholds
    plt.axhline(y=0.01, color='red', linestyle='--', linewidth=1)
    plt.text(gamma_bar_db[-10], 0.015, 'Pout = 0.01 (1%)', color='red')
    
    plt.axhline(y=0.001, color='orange', linestyle='--', linewidth=1)
    plt.text(gamma_bar_db[-10], 0.0015, 'Pout = 0.001 (0.1%)', color='orange')
    
    # Formatting
    plt.xlabel('Average SNR $\\bar{\\gamma}$ (dB)', fontsize=14)
    plt.ylabel('Outage Probability $P_{out}$', fontsize=14)
    plt.title('Outage Probability vs Average SNR (Threshold SNR $\\gamma_{th}$ = 1)', fontsize=16)
    plt.grid(True, alpha=0.3, which='both')
    plt.legend(loc='best')
    
    # Set y-axis to logarithmic scale for better visualization
    plt.yscale('log')
    plt.ylim(1e-5, 1.0)
    
    # Add equation reference in the plot
    equation_text = r"$P_{out} = \frac{\rho^2 z^k m^m}{2(2\sigma^2A_0^2h_l^2\bar{\gamma})^{\frac{\rho^2}{2}} \Gamma(m)} \gamma_{th}^{\frac{\rho^2}{2}} \sum_{j=0}^{\infty} \frac{a_j K^j}{(j!)^2} I_{3,1}^{3,4}[\cdot]$"
    plt.figtext(0.5, 0.01, equation_text, ha='center', fontsize=11, 
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    # Save the plot
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    computation_time = (time.time() - start_time) / 60.0  # in minutes
    print(f"Outage probability vs gamma bar plot generated and saved as '{output_filename}' (time: {computation_time:.2f} minutes)")
    
    # Also create a version with both axes in log scale (just for y-axis since x is already in dB)
    plt.figure(figsize=(12, 8))
    
    # Plot again for log scale in y-axis
    for i, fog_condition in enumerate(fog_conditions):
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        z_calc = 3 / (beta * d)
        
        # For log scale x-axis, we'll need to use a different range
        gamma_bar_log_db = np.linspace(-20, 10, 100)  # Extended lower range for log scale
        gamma_bar_log_range = 10**(gamma_bar_log_db/10)
        
        outage_probs_log = []
        for gamma_bar in gamma_bar_log_range:
            parameters = {
                'm': m,
                'h_l': h_l,
                'A_0': A_0,
                'K': K,
                'rho': rho,
                'z': z_calc,
                'k': k,
                'gamma_bar': gamma_bar,
                'sigma_squared': sigma_squared
            }
            
            if use_exact:
                try:
                    outage_prob = exact_cdf_function(gamma_th, parameters)
                except Exception:
                    outage_prob = cdf_simplified(gamma_th, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
            else:
                outage_prob = cdf_simplified(gamma_th, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
                
            outage_probs_log.append(outage_prob)
        
        plt.plot(gamma_bar_log_db, outage_probs_log, color=colors[color_idx], 
                 linestyle=linestyles[style_idx],
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    # Add reference lines
    plt.axhline(y=0.01, color='red', linestyle='--', linewidth=1)
    plt.text(-5, 0.015, 'Pout = 0.01 (1%)', color='red')
    
    plt.axhline(y=0.001, color='orange', linestyle='--', linewidth=1)
    plt.text(-5, 0.0015, 'Pout = 0.001 (0.1%)', color='orange')
    
    # Formatting
    plt.xlabel('Average SNR $\\bar{\\gamma}$ (dB)', fontsize=14)
    plt.ylabel('Outage Probability $P_{out}$', fontsize=14)
    plt.title('Outage Probability vs Average SNR (Log Scale, $\\gamma_{th}$ = 1)', fontsize=16)
    plt.grid(True, alpha=0.3, which='both')
    plt.legend(loc='best')
    
    # Set log scale for y-axis
    plt.yscale('log')
    plt.ylim(1e-5, 1.0)
    
    # Add equation reference
    plt.figtext(0.5, 0.01, equation_text, ha='center', fontsize=11, 
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
    
    plt.tight_layout(rect=[0, 0.05, 1, 0.95])
    plt.savefig(output_filename.replace('.png', '_full_log.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Additional log-scale plot saved as '{output_filename.replace('.png', '_full_log.png')}'")


def plot_outage_curves(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d, 
                       sigma_squared=None, use_exact=False, output_filename="outage_plot.png"):
    """
    Generate an outage probability plot using the CDF data.
    
    This plot shows the relationship between:
    - X-axis: Cumulative Outage Probability (%)
    - Y-axis: Available Capacity / Load (MW)
    
    Parameters:
    -----------
    m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d:
        Parameter values used in the CDF calculations
    sigma_squared: Optional, calculated from K if not provided
    use_exact: Whether to use exact CDF calculation (slower but more accurate)
    output_filename: Filename to save the plot
    """
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.ticker import PercentFormatter
    import time
    
    # Calculate sigma_squared if not provided
    if sigma_squared is None:
        sigma_squared = 1.0 / (1.0 + K)
    
    # Create figure
    plt.figure(figsize=(12, 8))
    
    # Define color map and line styles for different curves
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    linestyles = ['-', '--', '-.', ':']
    
    # Define capacity/load levels in MW
    # Using gamma values as proxy for capacity/load levels (scaled for visualization)
    max_capacity = 100  # Maximum system capacity in MW
    gamma_range = np.linspace(0, 10, 100)
    capacity_levels = gamma_range * (max_capacity/10)  # Scale gamma to MW
    
    # Track computation time
    start_time = time.time()
    
    # Plot for different fog conditions (representing different system states)
    fog_conditions = list(k_values.keys())
    for i, fog_condition in enumerate(fog_conditions):
        print(f"Processing outage curve for condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        color_idx = i % len(colors)
        style_idx = i % len(linestyles)
        
        # Get specific parameters for this condition
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        z_calc = 3 / (beta * d)
        
        # Use middle m value (represents a specific system configuration)
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Setup parameters for CDF calculation
        parameters = {
            'm': m,
            'h_l': h_l,
            'A_0': A_0,
            'K': K,
            'rho': rho,
            'z': z_calc,
            'k': k,
            'gamma_bar': gamma_bar,
            'sigma_squared': sigma_squared
        }
        
        # Calculate CDF values (represent cumulative outage probabilities)
        if use_exact:
            # Use exact calculation method
            try:
                cdf_values = [exact_cdf_function(gamma, parameters) for gamma in gamma_range]
            except Exception as e:
                print(f"Exact computation failed: {e}. Falling back to simplified method.")
                cdf_values = [cdf_simplified(gamma, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared) 
                             for gamma in gamma_range]
        else:
            # Use simplified/faster calculation method
            if HAS_CUPY:
                cdf_values = calculate_gpu_cdf_batch(gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
            else:
                cdf_values = parallel_cdf_calculation(
                    gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                    use_exact=False
                )
        
        # Convert to percentage for better visualization
        outage_probability = np.array(cdf_values) * 100
        
        # Plot the outage curve as a step function (traditional representation)
        plt.step(outage_probability, capacity_levels, color=colors[color_idx], 
                 linestyle=linestyles[style_idx], where='post',
                 label=f'{fog_condition} (k={k}, β={beta})')
    
    # Create a simulated Load Duration Curve (LDC)
    # In real applications, this would come from actual load data
    # Here we're creating a synthetic curve for demonstration
    
    # Generate LDC data points (using inverse exponential curve as an example)
    x_ldc = np.linspace(0, 100, 200)  # Probability points (0-100%)
    # Exponential decay model for load duration (higher loads have lower probability)
    y_ldc = max_capacity * np.exp(-0.03 * x_ldc)
    
    # Plot the Load Duration Curve
    plt.plot(x_ldc, y_ldc, 'k-', linewidth=2.5, label='Load Duration Curve')
    
    # Add a vertical line at peak load for reference
    peak_load = max(y_ldc)
    plt.axhline(y=peak_load, color='red', linestyle='--', linewidth=1)
    plt.text(2, peak_load + 2, f'Peak Load = {peak_load:.1f} MW', color='red')
    
    # Calculate reserve margin (assume 15% as standard)
    reserve_margin = 1.15 * peak_load
    if reserve_margin < max_capacity:
        plt.axhline(y=reserve_margin, color='blue', linestyle='--', linewidth=1)
        plt.text(2, reserve_margin + 2, f'Reserve Margin (15%) = {reserve_margin:.1f} MW', color='blue')
    
    # Find intersection points (where outage may occur)
    # This is simplified - in a real application you'd compute this more precisely
    plt.axvline(x=1.0, color='green', linestyle='--', linewidth=1)
    plt.text(1.5, max_capacity * 0.5, 'LOLP Threshold (1%)', color='green', rotation=90)
    
    # Formatting
    plt.xlabel('Cumulative Outage Probability (%)', fontsize=14)
    plt.ylabel('Available Capacity / Load (MW)', fontsize=14)
    plt.title('Outage Probability Plot with Load Duration Curve', fontsize=16)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper right')
    
    # Format x-axis as percentage
    plt.gca().xaxis.set_major_formatter(PercentFormatter())
    
    # Set axis limits
    plt.xlim(0, 100)
    plt.ylim(0, max_capacity * 1.1)
    
    # Add annotation explaining the plot
    annotation_text = (
        "Interpretation:\n"
        "• Where LDC intersects outage curves: potential loss of load\n"
        "• Area where LDC > outage curve: unserved energy\n"
        "• Steeper outage curve = lower system reliability\n"
        "• LOLP threshold: acceptable probability of load loss"
    )
    plt.annotate(annotation_text, xy=(0.02, 0.02), xycoords='figure fraction',
                 bbox=dict(boxstyle="round,pad=0.5", fc="lightyellow", alpha=0.8),
                 fontsize=9)
    
    # Save the plot
    plt.tight_layout()
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    computation_time = (time.time() - start_time) / 60.0  # in minutes
    print(f"Outage plot generated and saved as '{output_filename}' (time: {computation_time:.2f} minutes)")
    
    # Generate a complementary detailed plot showing reliability metrics
    generate_reliability_metrics_plot(m_values, fog_conditions, k_values, beta_values, 
                                     max_capacity, "reliability_metrics.png")

def generate_reliability_metrics_plot(m_values, fog_conditions, k_values, beta_values, 
                                     max_capacity, output_filename="reliability_metrics.png"):
    """
    Generate a supplementary plot showing reliability metrics derived from the outage analysis.
    
    Parameters:
    -----------
    m_values: List of m values used in the CDF calculations
    fog_conditions: List of fog condition names
    k_values, beta_values: Parameters for different fog conditions
    max_capacity: Maximum system capacity
    output_filename: Filename to save the plot
    """
    import numpy as np
    import matplotlib.pyplot as plt
    
    # Create figure with multiple subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Use middle m value for consistency
    m = m_values[1] if len(m_values) > 1 else m_values[0]
    
    # 1. Bar chart of LOLP (Loss of Load Probability) for different conditions
    # For demonstration, we'll simulate these values based on k values
    # In a real application, these would be calculated from the intersection of LDC and outage curves
    lolp_values = [0.1 / k for k in k_values.values()]  # Simulated values
    
    ax1.bar(fog_conditions, lolp_values, color='skyblue')
    ax1.set_title('Loss of Load Probability (LOLP)', fontsize=14)
    ax1.set_ylabel('Probability', fontsize=12)
    ax1.set_ylim(0, max(lolp_values) * 1.2)
    ax1.grid(axis='y', alpha=0.3)
    
    # Add threshold line
    threshold = 0.01  # 1% LOLP threshold
    ax1.axhline(y=threshold, color='red', linestyle='--')
    ax1.text(0, threshold * 1.05, 'Industry Standard Threshold (1%)', color='red')
    
    # Rotate x-axis labels for better readability
    plt.setp(ax1.get_xticklabels(), rotation=45, ha='right')
    
    # 2. Expected Unserved Energy (EUE) chart
    # Again, simulating based on beta values for demonstration
    eue_values = [beta / 500 for beta in beta_values.values()]  # Simulated values (MWh)
    
    ax2.bar(fog_conditions, eue_values, color='lightgreen')
    ax2.set_title('Expected Unserved Energy (EUE)', fontsize=14)
    ax2.set_ylabel('MWh per Year', fontsize=12)
    ax2.set_ylim(0, max(eue_values) * 1.2)
    ax2.grid(axis='y', alpha=0.3)
    
    # Rotate x-axis labels for better readability
    plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')
    
    # Add overall information
    plt.suptitle(f'System Reliability Metrics (m = {m})', fontsize=16)
    
    # Add explanatory text
    fig.text(0.5, 0.01, (
        "Notes: LOLP = Probability that load will exceed available generation capacity.\n"
        "EUE = Expected energy not supplied due to capacity shortages."
    ), ha='center', fontsize=10, bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(output_filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"Reliability metrics plot saved as '{output_filename}'")

def main():
    # Define parameters from the problem
    m_values = [2.3, 10.3, 20.3]
    h_l = 0.49933
    A_0 = 1
    gamma_bar = 1  # Default value for other plots
    K = 10
    rho = 3.38
    z = 4.343
    
    # Define fog condition parameters
    beta_values = {
        "Light Fog": 13.12,
        "Moderate Fog": 12.06,
        "Thick Fog": 23,
        "Dense Fog": 11.91
    }
    
    k_values = {
        "Light Fog": 2.32,
        "Moderate Fog": 5.49,
        "Thick Fog": 6,
        "Dense Fog": 36.05
    }
    
    d = 30  # distance in meters
    delta_values = [0.1, 0.5, 1]
    sigma_squared = 1 / (1 + K)
    
    # Log system information
    logging.info("===== SYSTEM INFORMATION =====")
    logging.info(f"Number of CPU cores: {mp_cpu.cpu_count()}")
    logging.info(f"GPU acceleration: {'Available' if HAS_CUPY else 'Not available'}")
    if HAS_CUPY:
        logging.info(f"GPU memory: {cp.cuda.Device().mem_info[1]/1e9:.2f} GB total")
    logging.info(f"mpmath precision: {mp.dps} digits")
    
    # Show system information
    print("System information:")
    print(f"  Number of CPU cores: {mp_cpu.cpu_count()}")
    print(f"  GPU acceleration: {'Available' if HAS_CUPY else 'Not available'}")
    if HAS_CUPY:
        print(f"  GPU memory: {cp.cuda.Device().mem_info[1]/1e9:.2f} GB total")
    print(f"  mpmath precision: {mp.dps} digits")
    print(f"  Logging to: {log_file}")
    print()
    
    # First generate the simplified (approximated) CDF plots
    print("\nGenerating simplified CDF plots...")
    logging.info("===== STARTING APPROXIMATE CDF CALCULATIONS =====")
    gamma_range = np.linspace(0, 10, 100)  # More points for smoother curves
    
    # Calculate total operations for unified progress tracking
    m_calculations = len(m_values) * len(gamma_range)
    fog_calculations = len(k_values) * len(gamma_range)
    delta_calculations = len(delta_values) * len(gamma_range)
    surface_points = 20 * 10  # For 3D surface points
    
    total_approximate_steps = m_calculations + fog_calculations + delta_calculations + surface_points
    
    # Create storage for results to plot later
    m_comparison_data = []
    fog_comparison_data = []
    delta_comparison_data = []
    
    # Create unified progress bar for all approximate calculations
    approx_progress = setup_progress_tracking(
        total_approximate_steps, 
        "Computing approximate CDFs"
    )
    
    # Track start time for performance logging
    approx_start_time = time.time()
    
    # Plot for different m values
    for i, m in enumerate(m_values):
        logging.info(f"Processing approximate CDF for m = {m} ({i+1}/{len(m_values)})")
        print(f"Processing approximate CDF for m = {m} ({i+1}/{len(m_values)})")
        
        # Use k value for Light Fog as default
        k = k_values["Light Fog"]
        
        # Use GPU acceleration if available
        if HAS_CUPY:
            logging.info(f"Using GPU acceleration for m={m}")
            cdf_values = calculate_gpu_cdf_batch(
                gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, 
                sigma_squared, progress_bar=approx_progress
            )
        else:
            # Use parallel CPU processing
            logging.info(f"Using parallel CPU processing for m={m}")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared,
                use_exact=False, progress_bar=approx_progress
            )
        
        # Store results for plotting
        m_comparison_data.append((m, cdf_values))
    
    # Plot for different fog conditions
    fog_conditions = list(k_values.keys())
    for i, fog_condition in enumerate(fog_conditions):
        logging.info(f"Processing approximate CDF for fog condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        print(f"Processing approximate CDF for fog condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
        
        k = k_values[fog_condition]
        beta = beta_values[fog_condition]
        
        # Recalculate z based on beta and d
        z_calc = 3 / (beta * d)
        
        # Use middle m value
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Use GPU acceleration if available
        if HAS_CUPY:
            logging.info(f"Using GPU acceleration for {fog_condition}")
            cdf_values = calculate_gpu_cdf_batch(
                gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, 
                sigma_squared, progress_bar=approx_progress
            )
        else:
            # Use parallel CPU processing
            logging.info(f"Using parallel CPU processing for {fog_condition}")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                use_exact=False, progress_bar=approx_progress
            )
        
        # Store results for plotting
        fog_comparison_data.append((fog_condition, k, beta, cdf_values))
    
    # Plot for different delta values
    for i, delta in enumerate(delta_values):
        logging.info(f"Processing approximate CDF for delta = {delta} ({i+1}/{len(delta_values)})")
        print(f"Processing approximate CDF for delta = {delta} ({i+1}/{len(delta_values)})")
        
        # Use Light Fog parameters and middle m value
        k = k_values["Light Fog"]
        beta = beta_values["Light Fog"]
        m = m_values[1] if len(m_values) > 1 else m_values[0]
        
        # Calculate modified z based on delta
        z_mod = z * (1 + delta)
        
        # Use GPU acceleration if available
        if HAS_CUPY:
            logging.info(f"Using GPU acceleration for delta={delta}")
            cdf_values = calculate_gpu_cdf_batch(
                gamma_range, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, 
                sigma_squared, progress_bar=approx_progress
            )
        else:
            # Use parallel CPU processing
            logging.info(f"Using parallel CPU processing for delta={delta}")
            cdf_values = parallel_cdf_calculation(
                gamma_range, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared,
                use_exact=False, progress_bar=approx_progress
            )
        
        # Store results for plotting
        delta_comparison_data.append((delta, cdf_values))
    
    # Calculate 3D surface data
    logging.info("Generating 3D surface data...")
    print("Generating 3D surface data...")
    gamma_mesh = np.linspace(min(gamma_range), max(gamma_range), 20)
    m_mesh = np.linspace(min(m_values), max(m_values), 10)
    gamma_grid, m_grid = np.meshgrid(gamma_mesh, m_mesh)
    z_grid = np.zeros_like(gamma_grid)
    
    # Use Light Fog k
    k = k_values["Light Fog"]
    
    # Calculate values for 3D surface
    for i in range(len(m_mesh)):
        for j in range(len(gamma_mesh)):
            z_grid[i, j] = cdf_simplified(gamma_grid[i, j], m_grid[i, j], h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
            approx_progress.update(1)
    
    # Calculate execution time
    approx_elapsed = time.time() - approx_start_time
    logging.info(f"Approximate CDF calculations completed in {approx_elapsed:.2f} seconds")
    
    # Close progress bar
    approx_progress.close()
    
    # Now create all the plots
    print("\nGenerating plots from calculated data...")
    logging.info("===== GENERATING PLOTS FROM APPROXIMATE DATA =====")
    colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k']
    linestyles = ['-', '--', '-.', ':']
    
    # Generate all the plots as before...
    # [Plotting code goes here - unchanged from your version]
    
    # Generate outage probability plot with varying gamma_bar
    print("\nGenerating outage probability plot vs gamma_bar with fixed gamma_th=1...")
    logging.info("===== GENERATING OUTAGE PROBABILITY VS GAMMA_BAR PLOTS =====")
    plot_outage_probability_vs_gamma_bar(m_values, h_l, A_0, K, rho, z, k_values, beta_values, d,
                                       sigma_squared=sigma_squared, use_exact=False)
    
    # The original outage probability plot (with varying gamma_th)
    print("\nGenerating original outage probability plot based on equation 14...")
    logging.info("===== GENERATING ORIGINAL OUTAGE PROBABILITY PLOTS =====")
    plot_outage_probability(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d,
                           sigma_squared=sigma_squared, use_exact=False)
    
    # Generate outage plot
    print("\nGenerating outage probability plot with load duration curve...")
    logging.info("===== GENERATING OUTAGE CURVE PLOTS =====")
    plot_outage_curves(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, beta_values, d,
                      sigma_squared=sigma_squared, use_exact=False)
    
    # Generate advanced 3D visualizations
    print("\nGenerating advanced 3D visualizations...")
    logging.info("===== GENERATING ADVANCED 3D VISUALIZATIONS =====")
    plot_advanced_3d_visualizations(m_values, h_l, A_0, K, rho, z, k_values, gamma_bar, 
                                  beta_values, d, delta_values, sigma_squared)
    
    # Test one computation to verify that the exact method works
    print("\nTesting exact computation mechanism...")
    logging.info("===== TESTING EXACT COMPUTATION =====")
    test_params = {
        'm': m_values[0],
        'h_l': h_l,
        'A_0': A_0,
        'K': K,
        'rho': rho,
        'z': z,
        'k': k_values["Light Fog"],
        'gamma_bar': gamma_bar,
        'sigma_squared': sigma_squared
    }
    
    start = time.time()
    # Test with timeout mechanism
    result = exact_cdf_function(1.0, test_params)
    test_time = time.time() - start
    
    print(f"Test computation took {test_time:.2f} seconds, result = {result}")
    logging.info(f"Test exact computation: result = {result}, time = {test_time:.2f} seconds")
    
    # Then generate the exact plots (which are computationally intensive)
    choice = input("\nDo you want to compute the exact CDF plots? This may take a very long time (y/n): ")
    logging.info(f"User chose {'to run' if choice.lower() == 'y' else 'not to run'} exact calculations")
    
    if choice.lower() == 'y':
        print("Computing exact CDF plots. This may take a long time...")
        logging.info("===== STARTING EXACT CDF CALCULATIONS =====")
        
        # Use fewer points for the computationally intensive exact method
        gamma_range_exact = np.linspace(0, 10, 30)
        
        # Create storage for exact results
        exact_m_comparison_data = []
        exact_fog_comparison_data = []
        exact_delta_comparison_data = []
        
        # Calculate total operations for exact calculations
        m_exact_calculations = len(m_values) * len(gamma_range_exact)
        fog_exact_calculations = len(k_values) * len(gamma_range_exact)
        delta_exact_calculations = len(delta_values) * len(gamma_range_exact)
        surface_exact_points = 10 * 6  # For 3D exact surface points
        
        total_exact_steps = m_exact_calculations + fog_exact_calculations + delta_exact_calculations + surface_exact_points
        
        # Create unified progress bar for exact calculations
        exact_progress = setup_progress_tracking(
            total_exact_steps, 
            "Computing exact CDFs"
        )
        
        # Track start time for performance logging
        exact_start_time = time.time()
        
        # Plot for different m values with exact calculation
        for i, m in enumerate(m_values):
            logging.info(f"Processing exact CDF for m = {m} ({i+1}/{len(m_values)})")
            print(f"Processing exact CDF for m = {m} ({i+1}/{len(m_values)})")
            
            k = k_values["Light Fog"]
            
            # Create parameter dictionary
            parameters = {
                'm': m,
                'h_l': h_l,
                'A_0': A_0,
                'K': K,
                'rho': rho,
                'z': z,
                'k': k,
                'gamma_bar': gamma_bar,
                'sigma_squared': sigma_squared
            }
            
            try:
                # Try to compute using parallel processing
                logging.info(f"Calculating exact CDF values for m={m} using parallel processing")
                cdf_values = parallel_cdf_calculation(
                    gamma_range_exact, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared,
                    use_exact=True, parameters=parameters, progress_bar=exact_progress
                )
            except Exception as e:
                error_msg = f"Parallel processing failed: {e}. Falling back to sequential calculation."
                logging.error(error_msg)
                print(error_msg)
                
                # Fall back to sequential processing
                cdf_values = []
                for gamma in gamma_range_exact:
                    try:
                        cdf_val = exact_cdf_function(gamma, parameters)
                    except Exception as e:
                        error_detail = f"Exact computation failed for gamma={gamma}, m={m}: {e}"
                        logging.error(error_detail)
                        print(error_detail)
                        cdf_val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
                    
                    cdf_values.append(cdf_val)
                    exact_progress.update(1)
            
            # Store results for plotting
            exact_m_comparison_data.append((m, cdf_values))
        
        # Continue with other exact calculations (fog conditions)
        for i, fog_condition in enumerate(fog_conditions):
            logging.info(f"Processing exact CDF for fog condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
            print(f"Processing exact CDF for fog condition: {fog_condition} ({i+1}/{len(fog_conditions)})")
            
            k = k_values[fog_condition]
            beta = beta_values[fog_condition]
            z_calc = 3 / (beta * d)
            
            # Use middle m value
            m = m_values[1] if len(m_values) > 1 else m_values[0]
            
            # Create parameter dictionary
            parameters = {
                'm': m,
                'h_l': h_l,
                'A_0': A_0,
                'K': K,
                'rho': rho,
                'z': z_calc,
                'k': k,
                'gamma_bar': gamma_bar,
                'sigma_squared': sigma_squared
            }
            
            try:
                # Try to compute using parallel processing
                logging.info(f"Calculating exact CDF values for {fog_condition} using parallel processing")
                cdf_values = parallel_cdf_calculation(
                    gamma_range_exact, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared,
                    use_exact=True, parameters=parameters, progress_bar=exact_progress
                )
            except Exception as e:
                error_msg = f"Parallel processing failed: {e}. Falling back to sequential calculation."
                logging.error(error_msg)
                print(error_msg)
                
                # Fall back to sequential processing
                cdf_values = []
                for gamma in gamma_range_exact:
                    try:
                        cdf_val = exact_cdf_function(gamma, parameters)
                    except Exception as e:
                        error_detail = f"Exact computation failed for {fog_condition}, gamma={gamma}: {e}"
                        logging.error(error_detail)
                        print(error_detail)
                        cdf_val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z_calc, k, gamma_bar, sigma_squared)
                    
                    cdf_values.append(cdf_val)
                    exact_progress.update(1)
            
            # Store results for plotting
            exact_fog_comparison_data.append((fog_condition, k, beta, cdf_values))
        
        # Calculate for different delta values with exact calculation
        for i, delta in enumerate(delta_values):
            logging.info(f"Processing exact CDF for delta = {delta} ({i+1}/{len(delta_values)})")
            print(f"Processing exact CDF for delta = {delta} ({i+1}/{len(delta_values)})")
            
            # Use Light Fog parameters and middle m value
            k = k_values["Light Fog"]
            m = m_values[1] if len(m_values) > 1 else m_values[0]
            
            # Calculate modified z based on delta
            z_mod = z * (1 + delta)
            
            # Create parameter dictionary
            parameters = {
                'm': m,
                'h_l': h_l,
                'A_0': A_0,
                'K': K,
                'rho': rho,
                'z': z_mod,
                'k': k,
                'gamma_bar': gamma_bar,
                'sigma_squared': sigma_squared
            }
            
            try:
                # Try to compute using parallel processing
                logging.info(f"Calculating exact CDF values for delta={delta} using parallel processing")
                cdf_values = parallel_cdf_calculation(
                    gamma_range_exact, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared,
                    use_exact=True, parameters=parameters, progress_bar=exact_progress
                )
            except Exception as e:
                error_msg = f"Parallel processing failed: {e}. Falling back to sequential calculation."
                logging.error(error_msg)
                print(error_msg)
                
                # Fall back to sequential processing
                cdf_values = []
                for gamma in gamma_range_exact:
                    try:
                        cdf_val = exact_cdf_function(gamma, parameters)
                    except Exception as e:
                        error_detail = f"Exact computation failed for delta={delta}, gamma={gamma}: {e}"
                        logging.error(error_detail)
                        print(error_detail)
                        cdf_val = cdf_simplified(gamma, m, h_l, A_0, K, rho, z_mod, k, gamma_bar, sigma_squared)
                    
                    cdf_values.append(cdf_val)
                    exact_progress.update(1)
            
            # Store results for plotting
            exact_delta_comparison_data.append((delta, cdf_values))
        
        # Calculate 3D surface data with exact CDF
        logging.info("Generating exact 3D surface data...")
        print("Generating exact 3D surface data...")
        gamma_mesh_exact = np.linspace(min(gamma_range_exact), max(gamma_range_exact), 10)
        m_mesh_exact = np.linspace(min(m_values), max(m_values), 6)
        gamma_grid_exact, m_grid_exact = np.meshgrid(gamma_mesh_exact, m_mesh_exact)
        z_grid_exact = np.zeros_like(gamma_grid_exact)
        
        # Use Light Fog k
        k = k_values["Light Fog"]
        
        # Calculate values for 3D surface with exact method
        for i in range(len(m_mesh_exact)):
            for j in range(len(gamma_mesh_exact)):
                m_val = m_grid_exact[i, j]
                gamma_val = gamma_grid_exact[i, j]
                
                # Create parameter dictionary
                parameters = {
                    'm': m_val,
                    'h_l': h_l,
                    'A_0': A_0,
                    'K': K,
                    'rho': rho,
                    'z': z,
                    'k': k,
                    'gamma_bar': gamma_bar,
                    'sigma_squared': sigma_squared
                }
                
                try:
                    z_grid_exact[i, j] = exact_cdf_function(gamma_val, parameters)
                except Exception as e:
                    error_detail = f"Error in exact computation for 3D surface at m={m_val}, gamma={gamma_val}: {e}"
                    logging.error(error_detail)
                    print(error_detail)
                    z_grid_exact[i, j] = cdf_simplified(gamma_val, m_val, h_l, A_0, K, rho, z, k, gamma_bar, sigma_squared)
                
                exact_progress.update(1)
        
        # Calculate execution time
        exact_elapsed = time.time() - exact_start_time
        logging.info(f"Exact CDF calculations completed in {exact_elapsed:.2f} seconds")
        
        # Close progress bar
        exact_progress.close()
        
        # Now create all the exact plots
        print("\nGenerating exact plots from calculated data...")
        logging.info("===== GENERATING PLOTS FROM EXACT DATA =====")
        
        logging.info("All exact plots have been generated successfully")
        print("Exact plots have been generated and saved.")
        print("Note: These calculations use the exact Fox H-function formula,")
        print("which is computationally intensive and may encounter numerical issues.")
    else:
        print("Skipping exact CDF plots.")
    
    # Log completion
    logging.info("===== ALL PROCESSING COMPLETED SUCCESSFULLY =====")
    print("\nAll processing completed successfully!")
    print(f"Full execution log available in: {log_file}")


if __name__ == "__main__":
    main()

GPU acceleration not available, using CPU only
System information:
  Number of CPU cores: 20
  GPU acceleration: Not available
  mpmath precision: 50 digits
  Logging to: cdf_calculation_20250519_111018.log


Generating simplified CDF plots...


Computing approximate CDFs:   0%|          | 0/1200 [00:00<?, ?it/s]

Processing approximate CDF for m = 2.3 (1/3)
Processing approximate CDF for m = 10.3 (2/3)
Processing approximate CDF for m = 20.3 (3/3)
Processing approximate CDF for fog condition: Light Fog (1/4)
Processing approximate CDF for fog condition: Moderate Fog (2/4)
Processing approximate CDF for fog condition: Thick Fog (3/4)
Processing approximate CDF for fog condition: Dense Fog (4/4)
Processing approximate CDF for delta = 0.1 (1/3)
Processing approximate CDF for delta = 0.5 (2/3)
Processing approximate CDF for delta = 1 (3/3)
Generating 3D surface data...

Generating plots from calculated data...

Generating outage probability plot vs gamma_bar with fixed gamma_th=1...


Processing fog conditions:   0%|          | 0/4 [00:00<?, ?it/s]

Processing outage probability for condition: Light Fog (1/4)


...

Computing for Dense Fog:   0%|          | 0/100 [00:00<?, ?it/s]

Outage probability vs gamma bar plot generated and saved as 'outage_probability_vs_gamma_bar.png' (time: 0.01 minutes)
Additional log-scale plot saved as 'outage_probability_vs_gamma_bar_full_log.png'

Generating original outage probability plot based on equation 14...
Processing outage probability for condition: Light Fog (1/4)
Processing outage probability for condition: Moderate Fog (2/4)
Processing outage probability for condition: Thick Fog (3/4)
Processing outage probability for condition: Dense Fog (4/4)
Outage probability plot generated and saved as 'outage_probability_plot.png' (time: 0.02 minutes)
Additional log-scale plot saved as 'outage_probability_plot_log_scales.png'

Generating outage probability plot with load duration curve...
Processing outage curve for condition: Light Fog (1/4)
Processing outage curve for condition: Moderate Fog (2/4)
Processing outage curve for condition: Thick Fog (3/4)
Processing outage curve for condition: Dense Fog (4/4)
Outage plot generated 

Computing exact CDFs:   0%|          | 0/360 [00:00<?, ?it/s]

Processing exact CDF for m = 2.3 (1/3)
Processing exact CDF for m = 10.3 (2/3)
Processing exact CDF for m = 20.3 (3/3)


Process ForkPoolWorker-470:
Process ForkPoolWorker-461:
Process ForkPoolWorker-469:
Process ForkPoolWorker-465:
Process ForkPoolWorker-464:
Process ForkPoolWorker-463:
Process ForkPoolWorker-467:
Process ForkPoolWorker-457:
Process ForkPoolWorker-475:
Process ForkPoolWorker-460:
Process ForkPoolWorker-462:
Process ForkPoolWorker-459:
Process ForkPoolWorker-472:
Process ForkPoolWorker-458:
Process ForkPoolWorker-466:
Process ForkPoolWorker-468:
Process ForkPoolWorker-473:
Process ForkPoolWorker-471:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceb

In [ ]:
# !pip install numpy matplotlib scipy mpmath tqdm

In [ ]:
# ! pip install -U jupyter ipywidgets
# ! jupyter nbextension enable --py widgetsnbextension
